<a href="https://colab.research.google.com/github/mariya-goltsova/Parser/blob/main/working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv")

In [3]:
df.sample(10)

,headlines,text
253,This is the India team I fantasised as a playe...,"Speaking about the current Indian team, former..."
17440,1st ever India-Pak T20I was decided by bowl-ou...,India and Pakistan played their first-ever T20...
68860,Mama never sleeps: Serena's 4-week-old daughte...,The official Instagram handle of Serena Willia...
97782,"After Karnataka, Haryana to replicate Amma can...",A team of officials from Haryana visited Amma ...
4743,"Told Suzuki, monkeys will leave on opening 'Ma...",Maruti Suzuki Chairman RC Bhargava said at a c...
26387,Can't silence me: Yogendra on tax raid at sist...,Alleging that the Income Tax department carrie...
75409,Toilet- Ek Prem Katha mints â¹51.45 crore in ...,The Akshay Kumar and Bhumi Pednekar starrer 'T...
17024,'Mard Ko Dard Nahi Hota' wins award at Toronto...,Radhika Madan starrer 'Mard Ko Dard Nahi Hota'...
94839,ISIS executes 33 in Syria in largest mass kill...,The ISIS executed 33 people in Syria's al-Maya...
69621,Pakistan-origin entrepreneurs' startup wins $1...,Pakistan-origin entrepreneurs' startup Roshni ...


In [4]:
df["text"] = "summarize: "+df["text"]

In [5]:
df.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"summarize: Saurav Kant, an alumnus of upGrad a..."
1,Delhi techie wins free food from Swiggy for on...,summarize: Kunal Shah's credit card bill payme...
2,New Zealand end Rohit Sharma-led India's 12-ma...,summarize: New Zealand defeated India by 8 wic...
3,Aegon life iTerm insurance plan helps customer...,summarize: With Aegon Life iTerm Insurance pla...
4,"Have known Hirani for yrs, what if MeToo claim...",summarize: Speaking about the sexual harassmen...


In [6]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from datasets import load_dataset
dataset = load_dataset('ccdv/arxiv-summarization')

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
tiny_dataset = dataset['train'].select(list(range(0, 100)))
tiny_dataset_val = dataset['validation'].select(list(range(0, 100)))
text_len = []
summary_len=[]

In [9]:
df = pd.DataFrame(tiny_dataset)
df_val = pd.DataFrame(tiny_dataset_val)

df["article"] = "summarize: "+df["article"]
df_val["article"] = "summarize: "+df_val["article"]

In [10]:
df.head()

,article,abstract
0,summarize: additive models @xcite provide an i...,additive models play an important role in semi...
1,summarize: the leptonic decays of a charged ps...,"we have studied the leptonic decay @xmath0 , v..."
2,summarize: the transport properties of nonline...,"in 84 , 258 ( 2000 ) , mateos conjectured that..."
3,summarize: studies of laser beams propagating ...,the effect of a random phase diffuser on fluct...
4,summarize: the so - called `` nucleon spin cri...,with a special intention of clarifying the und...


In [11]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [12]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [13]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [14]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [15]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [16]:
def T5Trainer(dataframe_train, dataframe_val, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  #dataframe = dataframe[[source_text,target_text]]
  #display_df(dataframe.head(2))
  dataframe_val = dataframe_val[[source_text,target_text]]
  #display_df(dataframe_val.head(2))
  dataframe_train = dataframe_train[[source_text,target_text]]
  display_df(dataframe_val.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  #train_size = 0.8
  #train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  train_dataset = dataframe_train
  #val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  val_dataset = dataframe_val.reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  #console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [17]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}

In [18]:
T5Trainer(dataframe_train=df, dataframe_val = df_val, source_text="article", target_text="abstract", model_params=model_params, output_dir="outputs")

[00:53:05] [Model]: Loading t5-base...                                           <ipython-input-16-799f1b7f4955>:14
                                                                                                                   

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[00:53:16] [Data]: Reading data...                                               <ipython-input-16-799f1b7f4955>:25
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|  summarize: the interest in anchoring phenomena and    | we study the phase behavior of a nematic liquid crystal|
|   phenomena in confined nematic liquid crystals has    | confined between a flat substrate with strong anchoring|
| largely been driven by their potential use in liquid   |   and a patterned substrate whose structure and local  |
|               crystal display devices .                |   anchoring strength we vary . by first evaluating an  |
|  the twisted nematic liquid crystal cell serves as an  |  effective surface free energy function characterizing |
|                       example .                        | the patterned substrate we derive an expression for the|
|    it consists of a nematic liquid crystal confined    |  effective free energy of the confined nematic liquid  |
|between two parallel walls , both providing homogeneous |                        crystal .                       |
| planar anchoring but with mutually perpendicular easy  |       then we determine phase diagrams involving a     |
|               directions . in this case                |   homogeneous state in which the nematic director is   |
|  the orientation of the nematic director is tuned by   |  almost uniform and a hybrid aligned nematic state in  |
|  the application of an external electric or magnetic   |  which the orientation of the director varies through  |
|                        field .                         |                       the cell .                       |
|  a precise control of the surface alignment extending  |  direct minimization of the free energy functional were|
|  over large areas is decisive for the functioning of   |    performed in order to test the predictions of the   |
|                    such devices .                      |             effective free energy method .             |
|  most studies have focused on nematic liquid crystals  |   we find remarkably good agreement between the phase  |
| in contact with laterally uniform substrates . on the  |     boundaries calculated from the two approaches .    |
|         other hand substrate inhomogeneities           |  in addition the effective energy method allows one to |
|     arise rather naturally as a result of surface      |  determine the energy barriers between two states in a |
|             treatments such as rubbing .               |                bistable nematic device .               |
|  thus the nematic texture near the surface is in fact  |                                                        |
|                    non - uniform .                     |                                                        |
|    this non - uniformity , however , is smeared out    |                                                        |
| beyond a decay length proportional to the periodicity  |                                                        |
|               of the surface pattern .                 |                                                        |
| very often the thickness of the non - uniform surface  |                                                        |
|layer is considerably smaller than both the wavelength  |                                                        |
|of visible light and the thickness of the nematic cell  |                                                        |
|    , i.e. , the distance between the two confining     |                                                        |
|                    parallel walls                     

TRAIN Dataset: (100, 2)

...

Completed 10

[00:55:32] [Validation Completed.]                                              <ipython-input-16-799f1b7f4955>:101
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt